### Calculation of contractual state transitions

In [2]:
import numpy as np

In [39]:
# we start with a state transition matrix of the form
# records x state x state (to)
transition_ass_timestep_res_ts = np.array([

    [[0.9, 0.1],
     [0, 1.0]],

    [[0.8, 0.2],
     [0.1, 0.9]],
    
])
transition_ass_timestep_res_ts

array([[[0.9, 0.1],
        [0. , 1. ]],

       [[0.8, 0.2],
        [0.1, 0.9]]])

In [26]:
# the state transitions look like this
state_transitions_current_month = [(0, 1, np.array([0, 1]))]
state_transitions_current_month

[(0, 1, array([0, 1]))]

### solution 1: change the values

This means that for the first insured we will have a state transition 1->2 (or 0->1) to 
be applied.

In [28]:
from_state = state_transitions_current_month[0][0]
to_state = state_transitions_current_month[0][1]
insured_selector = state_transitions_current_month[0][2].astype(bool)

from_state, to_state, insured_selector

(0, 1, array([False,  True]))

In [29]:
# this must be changed
transition_ass_timestep_res_ts[insured_selector, :, to_state]

array([[0.2, 0.9]])

In [30]:
# by adding this:
transition_ass_timestep_res_ts[insured_selector, :, from_state]

array([[0.8, 0.1]])

In [31]:
# i.e.
transition_ass_timestep_res_ts[insured_selector, :, to_state] += transition_ass_timestep_res_ts[insured_selector, :, from_state]
transition_ass_timestep_res_ts[insured_selector, :, from_state] = 0

# after the modification:
transition_ass_timestep_res_ts

array([[[0.9, 0.1],
        [0. , 1. ]],

       [[0. , 1. ],
        [0. , 1. ]]])

### solution 2

we expand the state transition into a tensor and "multiply from the right"

In [62]:
num_records, num_states = 2, 2

T = np.zeros((num_records, num_states, num_states))
T

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [63]:
# make set diagonals
for k in range(T.shape[1]):
    T[:, k, k] = 1

T

array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]])

In [68]:
# adjust
for st in range(num_states):
    T[insured_selector, from_state, st] = st == to_state
T

array([[[1., 0.],
        [0., 1.]],

       [[0., 1.],
        [0., 1.]]])

In [69]:
# now multiply
transition_ass_timestep_res_ts_mod = np.einsum('rij,rjk->rik', transition_ass_timestep_res_ts, T)
transition_ass_timestep_res_ts_mod

array([[[0.9, 0.1],
        [0. , 1. ]],

       [[0. , 1. ],
        [0. , 1. ]]])